In [85]:
%pip install -q ucimlrepo
%pip install -q sklearn
%pip install -q semopy
%pip install -q tableone

from ucimlrepo import fetch_ucirepo
import pandas as pd

# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

adult_df = pd.concat([X,y], axis=1)
adult_df.rename(lambda x: x.replace('-','_'), axis=1, inplace=True)

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import semopy
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,\
 recall_score, roc_auc_score

def get_causal_model_params(X_train, y_train, y_pred_proba, protected_attribute):
  '''
    Trains a semopy causal model on the given training data and predictions,\
     for the following linear causal model:\
      y_pred ~ beta0 + beta1*y_true + beta2*protected_attribute,\
     to identify the causal relationship between the protected attribute\
      and the predicted outcome.

     Inputs
       X_train: training features
       y_train: training labels
       y_pred_proba: predicted probabilities
       protected_attribute: name of the protected attribute

     Outputs
       beta2: coefficient of the causal relationship between\
        the protected attribute and the predicted outcome
       beta2_pvalue: p-value of the causal relationship
  '''
  causal_features = pd.DataFrame()
  causal_features['protected_attribute'] = X_train[protected_attribute]
  causal_features['y_true'] = y_train
  causal_features['y_pred'] = y_pred_proba

  model_desc='''
    y_true ~ protected_attribute
    y_pred ~ y_true + protected_attribute
  '''

  causal_model = semopy.Model(model_desc)
  causal_model.fit(causal_features)
  causal_params = causal_model.inspect()

  print(causal_params.to_markdown())

  # Retrieve the coefficients of the causal model
  beta2 = causal_params.loc[(causal_params.rval == "protected_attribute") & 
                            (causal_params.lval == "y_pred"),'Estimate'].values[0]
  beta2_pvalue = causal_params.loc[(causal_params.rval == "protected_attribute") & 
                            (causal_params.lval == "y_pred"),'p-value'].values[0]

  return [beta2, beta2_pvalue]

def get_perf_metrics(y_true, y_pred, y_pred_proba):
  '''
    Calculates the performance metrics for a given set of predictions.

    Inputs
      y_true: true labels
      y_pred: predicted labels
      y_pred_proba: predicted probabilities

    Outputs
      accuracy: accuracy score
      FNR: False Negative Rate
  '''
  accuracy = accuracy_score(y_true, y_pred)
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  FNR = fn / (fn + tp)

  return [accuracy, FNR]

# Data Pre-Processing

In [87]:
from tableone import TableOne

# Reduce dimensionality of categorical features before one-hot encoding
# native_country: bucket rare countries in 'other', handle na and ? as unknown
country_counts = adult_df['native_country'].value_counts(normalize=True)
rare_countries = country_counts[country_counts <= 0.01]

adult_df['native_country_grouped'] = adult_df['native_country'].replace(rare_countries.index, 'other')

adult_df['native_country_grouped'] = adult_df['native_country_grouped'].replace('?','unknown').fillna('Unknown')

# occupation: handle na and ? as unknown, categorise occupations
adult_df['occupation_grouped'] = adult_df['occupation'].replace('?', 'Unknown').fillna('Unknown')
occ_map = {
    'Prof-specialty': 'Professional', 'Exec-managerial': 'Professional',
    'Adm-clerical': 'Service_Admin', 'Sales': 'Service_Admin', 
    'Tech-support': 'Service_Admin', 'Protective-serv': 'Service_Admin',
    'Craft-repair': 'Blue_Collar', 'Machine-op-inspct': 'Blue_Collar', 
    'Transport-moving': 'Blue_Collar', 'Handlers-cleaners': 'Blue_Collar', 
    'Farming-fishing': 'Blue_Collar',
    'Other-service': 'Other', 'Priv-house-serv': 'Other', 'Armed-Forces': 'Other','Unknown':'Unknown'
}
adult_df['occupation_grouped'] = adult_df['occupation_grouped'].map(occ_map).fillna('Unknown')

# workclass: handle na and ? as unknown, categorise workclasses
adult_df['workclass_grouped'] = adult_df['workclass'].replace('?', 'Unknown').fillna('Unknown')
adult_df.loc[adult_df['workclass_grouped'].str.contains('gov'), 'workclass_grouped'] = 'Gov'
adult_df.loc[adult_df['workclass_grouped'].str.contains('Self-emp'), 'workclass_grouped'] = 'Self-emp'
adult_df.loc[adult_df['workclass_grouped'].str.contains('Never'), 'workclass_grouped'] = 'Without-pay'

# income label and sex feature
adult_df['income'] = adult_df['income'].map({'<=50K':0,'<=50K.':0,'>50K':1,'>50K.':1})
adult_df['sex'] = adult_df['sex'].map({'Male':1,'Female':0})

# Select features and labels
X = adult_df.drop(['workclass','education','occupation','native_country','fnlwgt','income'], axis=1)
cat_features = ['marital_status', 'relationship', 'race', 'sex', 'native_country_grouped', 'occupation_grouped', 'workclass_grouped']
num_features = ['age','capital_gain','capital_loss','hours_per_week']

y = adult_df['income']

# Baseline statistics
table1 = TableOne(adult_df,
                groupby='sex',
                continuous=num_features,
                categorical=cat_features)
print(table1)



                                                    Grouped by sex                                                  
                                                           Missing          Overall               0                1
n                                                                             48842           16192            32650
age, mean (SD)                                                   0      38.6 (13.7)     36.9 (14.1)      39.5 (13.4)
marital_status, n (%)         Divorced                                  6633 (13.6)     4001 (24.7)       2632 (8.1)
                              Married-AF-spouse                            37 (0.1)        25 (0.2)         12 (0.0)
                              Married-civ-spouse                       22379 (45.8)     2480 (15.3)     19899 (60.9)
                              Married-spouse-absent                       628 (1.3)       304 (1.9)        324 (1.0)
                              Never-married                     

# Model Training

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np

# Training / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Set up and train the pipeline
preprocessor = ColumnTransformer(
  transformers = [('num', StandardScaler(), num_features),
                  ('cat', OneHotEncoder(drop='first'), cat_features)]
  )
pipeline_steps = [('preprocessor', preprocessor), ('lg', LogisticRegression(max_iter=1000))]

pred_model = Pipeline(pipeline_steps)

pred_model.fit(X_train, y_train)

# Predicted outcome probabilities
y_pred_prob_train = pred_model.predict_proba(X_train)[:,1]
y_pred_prob_test = pred_model.predict_proba(X_test)[:,1]

# Apply classification threshold corresponding to 
# the prevalence of the negative class in the training set
neg_class_prevalence = 1 - y_train.sum()/len(y_train)
class_threshold = np.quantile(y_pred_prob_train, neg_class_prevalence)

y_pred_train = (y_pred_prob_train >= class_threshold).astype(int)
y_pred_test = (y_pred_prob_test >= class_threshold).astype(int)


## Performance and fairness of the baseline model

In [122]:
# Global performance of the baseline model
accuracy_train, FNR_train = get_perf_metrics(y_train, y_pred_train, y_pred_prob_train)
accuracy_test, FNR_test = get_perf_metrics(y_test, y_pred_test, y_pred_prob_test)

# Stratified performance
audit_df = X_test.copy()
audit_df['y_true'] = y_test
audit_df['y_pred'] = y_pred_test
audit_df['y_pred_prob'] = y_pred_prob_test

male_df = audit_df[audit_df['sex'] == 1]
female_df = audit_df[audit_df['sex'] == 0]

accuracy_m, FNR_m = get_perf_metrics(
  male_df['y_true'],
  male_df['y_pred'],
  male_df['y_pred_prob'])

accuracy_f, FNR_f = get_perf_metrics(
  female_df['y_true'],
  female_df['y_pred'],
  female_df['y_pred_prob'])

equal_opp_disparity = FNR_m - FNR_f

print(f'---TRAINING PERF---\n\
Global accuracy: {accuracy_train:.3f}, \
Global False Negative Rate: {FNR_train:.2f}')
print('\n---TEST PERF---')
print(f'\nGlobal accuracy: {accuracy_test:.3f}, \
Global False Negative Rate: {FNR_test:.2f}')
print(f'\nFemale accuracy: {accuracy_f:.3f}, \
Female False Negative Rate: {FNR_f:.2f}, \
Female True Positive Rate: {(1-FNR_f):.2f}')
print(f'\nMale accuracy: {accuracy_m:.3f}, \
Male False Negative Rate: {FNR_m:.2f}, \
Male True Positive Rate: {(1-FNR_m):.2f}')
print(f'\nEqual Opportunity Difference: {equal_opp_disparity:.2f}')


---TRAINING PERF---
Global accuracy: 0.836, Global False Negative Rate: 0.34

---TEST PERF---

Global accuracy: 0.838, Global False Negative Rate: 0.34

Female accuracy: 0.923, Female False Negative Rate: 0.43, Female True Positive Rate: 0.57

Male accuracy: 0.797, Male False Negative Rate: 0.33, Male True Positive Rate: 0.67

Equal Opportunity Difference: -0.10


# Causal-based bias detection and mitigation

In [110]:
# Causal Model Analysis on the training set
# Considering the following causal model:
# y_pred = beta0 + beta1*y_true + beta2*a
# where a is the protected attribute, i.e. sex
beta2, beta2_pvalue = get_causal_model_params(X_train, y_train, y_pred_prob_train, 'sex')

print(f'\nCausal path between protected attribute (sex) and prediction: {round(beta2,3)} (P-value = {beta2_pvalue})')

|    | lval   | op   | rval                |   Estimate |    Std. Err |   z-value |   p-value |
|---:|:-------|:-----|:--------------------|-----------:|------------:|----------:|----------:|
|  0 | y_true | ~    | protected_attribute |   0.192353 | 0.00490262  |   39.2347 |         0 |
|  1 | y_pred | ~    | y_true              |   0.373796 | 0.00268793  |  139.064  |         0 |
|  2 | y_pred | ~    | protected_attribute |   0.120373 | 0.00243928  |   49.3478 |         0 |
|  3 | y_true | ~~   | y_true              |   0.174466 | 0.00136393  |  127.914  |         0 |
|  4 | y_pred | ~~   | y_pred              |   0.041249 | 0.000322475 |  127.914  |         0 |

Causal path between protected attribute (sex) and prediction: 0.12 (P-value = 0.0)
